# Librerías

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import seaborn as sns

%matplotlib inline

# Predicciones en csv

In [2]:
preds_lightgbm = pd.read_csv('../predictions/week2/preds_lightgbm_pw.csv')
preds_catboost = pd.read_csv('../predictions/week2/preds_catboost_pw.csv')
preds_xgboost = pd.read_csv('../predictions/week2/preds_xgboost_pw.csv')

In [3]:
full_preds = preds_lightgbm.join(preds_catboost.set_index('Unnamed: 0'), on='Unnamed: 0')\
.join(preds_xgboost.set_index('Unnamed: 0'), on='Unnamed: 0')
full_preds.head(10)

,Unnamed: 0,predicciones_lightgbm_pw,predicciones_catboost_pw,predicciones_xgboost_pw
0,0,30,89,29
1,1,80,132,62
2,2,15,23,16
3,3,5,8,6
4,4,35,82,22
5,5,115,193,72
6,6,63,113,47
7,7,1,1,1
8,8,4,4,8
9,9,43,100,34


In [4]:
full_preds['media'] = (full_preds['predicciones_lightgbm_pw'] +
                       full_preds['predicciones_catboost_pw'] +
                       full_preds['predicciones_xgboost_pw'])/3
full_preds.head(10)

,Unnamed: 0,predicciones_lightgbm_pw,predicciones_catboost_pw,predicciones_xgboost_pw,media
0,0,30,89,29,49.333333
1,1,80,132,62,91.333333
2,2,15,23,16,18.000000
3,3,5,8,6,6.333333
4,4,35,82,22,46.333333
5,5,115,193,72,126.666667
6,6,63,113,47,74.333333
7,7,1,1,1,1.000000
8,8,4,4,8,5.333333
9,9,43,100,34,59.000000


In [5]:
y_preds = [int(round(x)) for x in full_preds['media'].values.tolist()]

In [6]:
y_preds[:20]

[49, 91, 18, 6, 46, 127, 74, 1, 5, 59, 1, 23, 4, 0, 113, 128, 54, 24, 14, 89]

# Apuesta de bloque

In [7]:
product_blocks = pd.read_csv('../data/product_blocks.csv')
product_blocks.head(10)

,product_id,block_id
0,612967398,0
1,296892108,0
2,139541214,0
3,963923934,0
4,938230141,0
5,172045154,0
6,663552768,0
7,160621689,1
8,948976891,1
9,556017319,1


In [8]:
productos_por_bloque = product_blocks.groupby('block_id').count()['product_id']
productos_por_bloque.name = 'n_products'
productos_por_bloque.head()

block_id
0     7
1     7
2     7
3     6
4    10
Name: n_products, dtype: int64

In [9]:
product_blocks_n = product_blocks.join(productos_por_bloque, on='block_id', how='left')
product_blocks_n.head()

,product_id,block_id,n_products
0,612967398,0,7
1,296892108,0,7
2,139541214,0,7
3,963923934,0,7
4,938230141,0,7


In [10]:
test = pd.read_csv('../data/week2/final_test_pw.csv', usecols=['product_id', 'price'])

In [11]:
test.head()

,product_id,price
0,151926,25.95
1,213413,19.95
2,310130,12.95
3,455200,29.95
4,571044,15.95


In [12]:
predicciones = pd.DataFrame({'product_id': test.product_id,
                             'preds': y_preds,
                             'price':test.price,
                             'gain': y_preds * test.price})

predicciones = predicciones.sort_values('gain', ascending=False)
predicciones.head()

,product_id,preds,price,gain
330,37119458,1840,22.95,42228.00
3899,429109570,1113,29.95,33334.35
4101,450385036,1594,19.95,31800.30
1022,112446414,1065,25.95,27636.75
4230,466243134,1268,19.95,25296.60


In [13]:
predicciones.shape, test.shape

((9007, 4), (9007, 2))

In [14]:
predicciones_final = predicciones.join(product_blocks_n.set_index('product_id'),
                                       on='product_id', how='left').reset_index(drop=True)
predicciones_final.head()

,product_id,preds,price,gain,block_id,n_products
0,37119458,1840,22.95,42228.00,812,8
1,429109570,1113,29.95,33334.35,1412,5
2,450385036,1594,19.95,31800.30,2233,8
3,112446414,1065,25.95,27636.75,2306,3
4,466243134,1268,19.95,25296.60,2000,5


In [15]:
group_block_gain = predicciones_final.groupby('block_id').sum().sort_values('gain', ascending=False)['gain']
group_block_gain.name = 'gain_per_block'

In [16]:
preds_final = predicciones_final.join(group_block_gain, on='block_id')
preds_final = preds_final.sort_values('gain_per_block', ascending=False)
preds_final = preds_final.reset_index(drop=True)

In [17]:
preds_final.head()

,product_id,preds,price,gain,block_id,n_products,gain_per_block
0,37119458,1840,22.95,42228.00,812,8,44849.55
1,890876414,54,29.95,1617.30,812,8,44849.55
2,19599326,11,15.95,175.45,812,8,44849.55
3,679568934,64,12.95,828.80,812,8,44849.55
4,234797504,81,7.95,643.95,1580,8,35702.90


In [18]:
preds_final = preds_final.iloc[preds_final.block_id.drop_duplicates().index.values.tolist()]
preds_final = preds_final.sort_values('gain_per_block', ascending=False)

In [19]:
preds_final.head()

,product_id,preds,price,gain,block_id,n_products,gain_per_block
0,37119458,1840,22.95,42228.00,812,8,44849.55
4,234797504,81,7.95,643.95,1580,8,35702.90
12,275108971,25,22.95,573.75,1412,5,34207.60
15,797965294,85,25.95,2205.75,1143,10,32897.39
24,921904870,16,22.95,367.20,2233,8,32660.55


In [20]:
preds_final[preds_final.block_id==560]

,product_id,preds,price,gain,block_id,n_products,gain_per_block
56,231065399,63,25.95,1634.85,560,8,20603.45


In [21]:
preds_final['product_cumsum'] = preds_final.n_products.cumsum()
preds_final.head()

,product_id,preds,price,gain,block_id,n_products,gain_per_block,product_cumsum
0,37119458,1840,22.95,42228.00,812,8,44849.55,8
4,234797504,81,7.95,643.95,1580,8,35702.90,16
12,275108971,25,22.95,573.75,1412,5,34207.60,21
15,797965294,85,25.95,2205.75,1143,10,32897.39,31
24,921904870,16,22.95,367.20,2233,8,32660.55,39


In [22]:
bet_blocks = preds_final[preds_final.product_cumsum <= 100]
bet_blocks

,product_id,preds,price,gain,block_id,n_products,gain_per_block,product_cumsum
0,37119458,1840,22.95,42228.00,812,8,44849.55,8
4,234797504,81,7.95,643.95,1580,8,35702.90,16
12,275108971,25,22.95,573.75,1412,5,34207.60,21
15,797965294,85,25.95,2205.75,1143,10,32897.39,31
24,921904870,16,22.95,367.20,2233,8,32660.55,39
27,112446414,1065,25.95,27636.75,2306,3,27636.75,42
28,519589876,35,25.95,908.25,2000,5,26983.55,47
31,279854545,109,9.95,1084.55,487,8,25425.68,55
38,87049042,36,59.95,2158.20,2037,4,23744.55,59
41,686215283,1,69.95,69.95,387,8,22589.80,67


In [23]:
preds_final[['block_id', 'gain_per_block', 'n_products', 'product_cumsum']].to_csv('../bets/week2_bets_pw.csv', index=None)

### Posibles bloques para la apuesta final:

In [25]:
bet_blocks.block_id

0      812
4     1580
12    1143
21     388
26     442
32    2735
37     487
44    2574
52     530
59    1747
Name: block_id, dtype: int64